# 🥔 02_features_clustering.ipynb
## Extracción de características y clustering para limpieza de dataset de hojas de papa
Este notebook realiza:
1. Extracción de embeddings con MobileNetV2.
2. Clustering dentro de cada clase para detectar y eliminar imágenes atípicas (ruido).
3. Guarda los embeddings limpios para entrenamiento posterior de un clasificador.

Se asume que las imágenes ya fueron preprocesadas (resize, contraste condicional, normalización).


In [17]:
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.cluster import DBSCAN
import pickle
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

# --------------------------
data_dir = Path("../data/2_data_resize")  # Dataset preprocesado
output_dir = Path("../data/4_features_embeddings")
output_dir.mkdir(parents=True, exist_ok=True)


## 1️⃣ Cargar MobileNetV2 preentrenado
Se usa MobileNetV2 sin la capa de clasificación (`include_top=False`) para extraer embeddings de cada imagen.


In [18]:
# Cargar modelo MobileNetV2 para extracción de features
model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224,224,3))

## 2️⃣ Función para leer y preprocesar imágenes
Convierte las imágenes a RGB, aplica `preprocess_input` de MobileNetV2.

In [19]:
def load_and_preprocess(img_path):
    img = cv2.imread(str(img_path))
    if img is None:
        return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    img = preprocess_input(img.astype('float32'))  # Escala -1 a 1
    return img


## 3️⃣ Extraer embeddings por clase
Se recorren todas las clases, se extraen embeddings y se almacenan para clustering.


In [20]:
embeddings_dict = {}
labels_dict = {}

for class_folder in sorted(os.listdir(data_dir)):
    class_path = data_dir / class_folder
    if not class_path.is_dir():
        continue

    embeddings = []
    image_files = []

    for img_file in tqdm(os.listdir(class_path), desc=f"Procesando {class_folder}"):
        img_path = class_path / img_file
        img = load_and_preprocess(img_path)
        if img is None:
            continue
        # Extraer embedding
        emb = model.predict(np.expand_dims(img, axis=0), verbose=0)
        embeddings.append(emb.flatten())
        image_files.append(img_file)

    embeddings = np.array(embeddings)
    embeddings_dict[class_folder] = embeddings
    labels_dict[class_folder] = image_files


Procesando Virus: 100%|██████████| 532/532 [03:40<00:00,  2.42it/s]


## 4️⃣ Clustering para eliminar outliers
Se aplica DBSCAN a los embeddings de cada clase. Los puntos etiquetados como `-1` se consideran outliers y se eliminan.


In [29]:
# ------------------------------
# BLOQUE: Clustering adaptativo por clase (garantiza retención mínima)
# ------------------------------
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import shutil

# Parámetros ajustables
PCA_DIM = 30
INITIAL_EPS = 1.5
EPS_MULT = 1.6       # factor para ir aumentando eps si se elimina demasiado
MAX_EPS = 80.0
MIN_SAMPLES = 3
MIN_KEEP_RATIO = 0.90   # queremos conservar al menos 90% por clase

# Carpetas para sospechosas
suspect_dir = output_dir / "suspect_images"
suspect_dir.mkdir(parents=True, exist_ok=True)

clean_embeddings = {}
clean_labels = {}
suspect_labels = {}

for class_name, embeddings in embeddings_dict.items():
    n_total = len(embeddings)
    if n_total == 0:
        clean_embeddings[class_name] = embeddings
        clean_labels[class_name] = []
        suspect_labels[class_name] = []
        print(f"{class_name}: 0 imágenes (skip).")
        continue

    if n_total < 5:
        # Si muy pocas imágenes, no clusterizamos automáticamente
        clean_embeddings[class_name] = embeddings
        clean_labels[class_name] = labels_dict[class_name]
        suspect_labels[class_name] = []
        print(f"{class_name}: {n_total} imágenes (<5) → no se clusteriza, se conservan todas.")
        continue

    # PCA reducción
    pca = PCA(n_components=min(PCA_DIM, n_total))  # no pedir más comps que muestras
    embeddings_reduced = pca.fit_transform(embeddings)

    # intento adaptativo con DBSCAN aumentando eps hasta cumplir MIN_KEEP_RATIO
    eps = INITIAL_EPS
    final_mask = None
    final_labels = None

    while True:
        clusterer = DBSCAN(eps=eps, min_samples=MIN_SAMPLES, metric='euclidean')
        cluster_labels = clusterer.fit_predict(embeddings_reduced)  # -1 = outlier
        mask = cluster_labels != -1
        kept = mask.sum()
        keep_ratio = kept / n_total

        # Si cumplimos la proporción mínima, aceptamos
        if keep_ratio >= MIN_KEEP_RATIO:
            final_mask = mask
            final_labels = cluster_labels
            break

        # Si eps ya demasiado grande, no eliminar nada (fallback seguro)
        if eps >= MAX_EPS:
            final_mask = np.ones(n_total, dtype=bool)  # conservar todo
            final_labels = np.zeros(n_total, dtype=int)  # todos en un cluster ficticio
            print(f"{class_name}: No se pudo alcanzar keep_ratio={MIN_KEEP_RATIO:.2f} incluso con eps={eps:.1f}. Conservando todo (fallback).")
            break

        # aumentar eps y reintentar
        eps *= EPS_MULT

    # Aplicar máscara final
    mask = final_mask
    kept_idx = np.where(mask)[0]
    suspect_idx = np.where(~mask)[0]

    clean_embeddings[class_name] = embeddings[kept_idx]
    clean_labels[class_name] = np.array(labels_dict[class_name])[kept_idx].tolist()
    suspect_labels[class_name] = np.array(labels_dict[class_name])[suspect_idx].tolist()

    # Reporte por clase
    print(f"{class_name}: {n_total} imágenes → {len(clean_labels[class_name])} limpias, {len(suspect_labels[class_name])} sospechosas (eps final={eps:.2f})")

    # Mover/copiar imágenes sospechosas a carpeta de revisión (no borra nada)
    class_suspect_dir = suspect_dir / class_name
    class_suspect_dir.mkdir(parents=True, exist_ok=True)
    for fname in suspect_labels[class_name]:
        src = data_dir / class_name / fname
        dst = class_suspect_dir / fname
        if src.exists():
            # copia por seguridad (si dataset grande, podrías cambiar a move)
            shutil.copy2(src, dst)

# Guardar resultados
with open(output_dir / "embeddings_clean.pkl", "wb") as f:
    pickle.dump(clean_embeddings, f)

with open(output_dir / "labels_clean.pkl", "wb") as f:
    pickle.dump(clean_labels, f)

with open(output_dir / "labels_suspect.pkl", "wb") as f:
    pickle.dump(suspect_labels, f)

print("Proceso completado. Embeddings/labels limpias y sospechosas guardadas. Revisa la carpeta 'suspect_images' para inspección manual.")


Bacteria: 569 imágenes → 569 limpias, 0 sospechosas (eps final=15.73)
Fungi: 748 imágenes → 748 limpias, 0 sospechosas (eps final=15.73)
Healthy: 201 imágenes → 198 limpias, 3 sospechosas (eps final=15.73)
Nematode: 68 imágenes → 67 limpias, 1 sospechosas (eps final=25.17)
Pest: 611 imágenes → 610 limpias, 1 sospechosas (eps final=15.73)
Phytopthora: 347 imágenes → 340 limpias, 7 sospechosas (eps final=15.73)
Virus: 532 imágenes → 531 limpias, 1 sospechosas (eps final=15.73)
Proceso completado. Embeddings/labels limpias y sospechosas guardadas. Revisa la carpeta 'suspect_images' para inspección manual.


## 5️⃣ Guardar embeddings y etiquetas limpias
Se guardan en formato pickle para entrenamiento posterior.


In [23]:
with open(output_dir / "embeddings_clean.pkl", "wb") as f:
    pickle.dump(clean_embeddings, f)

with open(output_dir / "labels_clean.pkl", "wb") as f:
    pickle.dump(clean_labels, f)

print("Embeddings y etiquetas limpias guardadas correctamente.")


Embeddings y etiquetas limpias guardadas correctamente.


## 6️⃣ Visualización de clusters y outliers
Aquí proyectaremos los embeddings de cada clase en 2D para ver cómo se agrupan y cuáles se consideran outliers.


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

for class_name, embeddings in clean_embeddings.items():
    if len(embeddings) < 5:
        continue

    # PCA a 2D
    pca = PCA(n_components=2)
    emb_2d = pca.fit_transform(embeddings_dict[class_name])  # usamos todos los embeddings originales para ver outliers
    cluster_labels = DBSCAN(eps=5.0, min_samples=2, metric='euclidean').fit_predict(embeddings_dict[class_name])

    # Dibujar
    plt.figure(figsize=(6,5))
    for label in set(cluster_labels):
        mask = cluster_labels == label
        if label == -1:
            # Outliers en rojo
            plt.scatter(emb_2d[mask,0], emb_2d[mask,1], c='red', label='Outlier', alpha=0.6)
        else:
            plt.scatter(emb_2d[mask,0], emb_2d[mask,1], alpha=0.6, label=f'Cluster {label}')
    plt.title(f'Clusters y outliers: {class_name}')
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    plt.legend()
    plt.show()


SyntaxError: unmatched ']' (301075031.py, line 79)